In [4]:
tech_stack = spark.sql(f"SELECT tech_stack FROM bing_lake_db.naukri_jobs WHERE lower(job_title) LIKE '%{job_titles}%'")

StatementMeta(, 1ccbddf2-f2a3-4b0e-abb0-72939e03c7f7, 6, Finished, Available, Finished)

In [5]:
display(tech_stack)

StatementMeta(, 1ccbddf2-f2a3-4b0e-abb0-72939e03c7f7, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 46beb0b7-bbda-4658-8d18-04d40501783e)

In [6]:
from pyspark.sql.functions import split, explode

# Split the string column by comma to create an array column
df_split = tech_stack.withColumn('tech_stack_array', split(tech_stack['tech_stack'], ','))

# Explode the array column into rows
df_exploded = df_split.select(explode(df_split['tech_stack_array']).alias('tech_stack'))

# Collect the values into a list
tech_stack = [row['tech_stack'] for row in df_exploded.collect()]

tech_stack=[x.lower() for x in tech_stack]

StatementMeta(, 1ccbddf2-f2a3-4b0e-abb0-72939e03c7f7, 8, Finished, Available, Finished)

In [7]:
import collections
from pyspark.sql.functions import col,lit
# Count occurrences
skill_counts = collections.Counter(tech_stack)

# Convert Counter to DataFrame
tech_stack = spark.createDataFrame(list(skill_counts.items()), ['skill', 'count'])

# Add a new column with a static value
tech_stack = tech_stack.withColumn('job_title', lit(f'{job_titles}'))

# Optionally, you can sort the DataFrame by count descending
tech_stack = tech_stack.orderBy(col('count').desc())

# Show the sorted DataFrame
tech_stack.show()

StatementMeta(, 1ccbddf2-f2a3-4b0e-abb0-72939e03c7f7, 9, Finished, Available, Finished)

+--------------------+-----+-------------+
|               skill|count|    job_title|
+--------------------+-----+-------------+
|              python|   56|data engineer|
|                 sql|   48|data engineer|
|    data engineering|   47|data engineer|
|             pyspark|   40|data engineer|
|                data|   31|data engineer|
|    data warehousing|   28|data engineer|
|       data modeling|   22|data engineer|
|     microsoft azure|   17|data engineer|
|               scala|   17|data engineer|
|                 etl|   16|data engineer|
|               spark|   16|data engineer|
|                 aws|   16|data engineer|
|                 gcp|   15|data engineer|
|  azure data factory|   15|data engineer|
|            big data|   14|data engineer|
|                hive|   14|data engineer|
|information techn...|   13|data engineer|
|               azure|   11|data engineer|
|            aws glue|   11|data engineer|
|       data engineer|   11|data engineer|
+----------

In [19]:
from pyspark.sql.utils import AnalysisException

try:
    table_name='bing_lake_db.tech_stack_count'
    tech_stack.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table Already Exists")
    table_name='bing_lake_db.tech_stack_count'
    #tech_stack_existing = spark.sql("SELECT tech_stack FROM bing_lake_db.naukri_jobs")
    tech_stack.createOrReplaceTempView("tech_stack_new")
    spark.sql(f"""
                MERGE INTO {table_name} AS target
                USING (
                    SELECT job_title, skill, SUM(count) AS count
                    FROM (
                        SELECT job_title, skill, CAST(count AS BIGINT) AS count
                        FROM tech_stack_new
                        ORDER BY count DESC
                    )
                    GROUP BY job_title, skill
                ) AS source
                ON target.job_title = source.job_title AND target.skill = source.skill
                WHEN MATCHED THEN UPDATE SET target.count = target.count + source.count
                WHEN NOT MATCHED THEN INSERT (job_title, skill, count) VALUES (source.job_title, source.skill, source.count)       
                """)

StatementMeta(, 1ccbddf2-f2a3-4b0e-abb0-72939e03c7f7, 21, Finished, Available, Finished)

Table Already Exists
